Jordan Picone's project

In [1]:
import json
import pandas as pd
import nltk
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle

In [2]:
def isRes(ID):
        if ID in res_tag:
                return True
        else:
                return False

def cuisine(cat):
        # finish these up before analysis
        # To find these labels I filtered all businesses tagged as restaurants and made a set of 
        # other tags associated with those restaurants. Out of those 343 I chose the labels manually
        american = ['American (New)', 'American', 'American (Traditional)', 'Southern']
        middle_east = ['Moroccan', 'Egyptian', 'Middle Eastern', 'Turkish', 'Halal', 'Pakistani', 'Persian/Iranian', 'Arabian',]
        latin_america = ['Latin American', 'Peruvian', 'Venezuelan', 'Salvadoran', 'Mexican', 'Brazilian', 'Colombian', 'Puerto Rican']
        east_asia = ['Asian', 'Taiwanese', 'Cambodian', 'Izakaya', 'Singaporean', 'Thai', 'Hong Kong Style Cafe', 'Korean', 'Japanese', 'Chinese', 'Laotian', 'Szechuan', 'Cantonese', 'Vietnamese']
        if cat == None:
            return None
        x = cat.split(', ')
        for i in x:
            if i in middle_east:
                return 'me'
            if i in american:
                return 'am'
            if i in latin_america:
                return 'la'
            if i in east_asia:
                return 'as'
            else:
                return None

Read in businesses and filter out the ones we want

In [3]:
fn = r'yelp_dataset\yelp_academic_dataset_business.json'
res = pd.read_json(fn, lines=True)
res['restaurant'] = res['categories'].map(lambda x: cuisine(x))
res = res.dropna()

In [4]:
res['restaurant'].value_counts()

as    2787
am    2327
la    1400
me     614
Name: restaurant, dtype: int64

In [5]:
res = res.dropna(subset = ['restaurant'])

In [6]:
res.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,restaurant
30,9sRGfSVEfLhN_km60YruTA,Apadana Restaurant,13071 Yonge Street,Richmond Hill,ON,L4E 1A5,43.947011,-79.454862,3.0,3,1,"{'Ambience': '{'touristy': False, 'hipster': F...","Persian/Iranian, Turkish, Middle Eastern, Rest...","{'Tuesday': '12:0-21:0', 'Wednesday': '12:0-21...",me
44,Ga2Bt7xfqoggTypWD5VpoQ,Amando's Bros,2602 W Southern Ave,Tempe,AZ,85282,33.393199,-111.976270,4.0,9,0,"{'Caters': 'False', 'RestaurantsGoodForGroups'...","Mexican, Restaurants","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",la
73,0QjROMVW9ACKjhSEfHqNCQ,Mi Mi Restaurant,688 Gerrard Street E,Toronto,ON,M4M 1Y3,43.666376,-79.348773,4.0,116,1,"{'RestaurantsTakeOut': 'True', 'Alcohol': 'u'b...","Vietnamese, Restaurants","{'Monday': '11:0-22:0', 'Tuesday': '11:0-22:0'...",as
94,Ir_QIzs-4o9ElOtiGuxJrw,Torteria Lupita,4601 Rue Notre-Dame O,Montréal,QC,H4C 1S5,45.473632,-73.588135,4.0,19,0,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Mexican, Sandwiches, Restaurants, Breakfast & ...","{'Tuesday': '12:0-20:0', 'Wednesday': '12:0-20...",la
137,t749-3jKKDyO8QH3VezUHA,Chao Phraya,50 Avenue Laurier O,Montréal,QC,H2T 2N4,45.522376,-73.593123,3.5,48,0,"{'RestaurantsReservations': 'True', 'Ambience'...","Thai, Restaurants","{'Tuesday': '17:0-22:0', 'Wednesday': '17:0-22...",as


In [7]:
# read in the reviews
fn = r'yelp_dataset\yelp_academic_dataset_review.json'
ChunkSize = 100000
# This is a weird way to do it, but it's necessary with such a huge file.
for chunk in pd.read_json(fn, chunksize=ChunkSize, lines=True):
    df = chunk
    break

In [8]:
res[['business_id', 'restaurant']]

,business_id,restaurant
30,9sRGfSVEfLhN_km60YruTA,me
44,Ga2Bt7xfqoggTypWD5VpoQ,la
73,0QjROMVW9ACKjhSEfHqNCQ,as
94,Ir_QIzs-4o9ElOtiGuxJrw,la
137,t749-3jKKDyO8QH3VezUHA,as
...,...,...
209315,eUi3O-8Gqh_nJ_ZhO-25gQ,la
209369,5VXlHFrYZZzj7zfZEsz5nQ,as
209376,cmsAFAymfVoBitSF8R6GKw,as
209388,9Q0fPWAjUweoFDk0kafuzQ,as


In [9]:
df = df.merge(res[['business_id', 'restaurant']], how='left')

In [10]:
# narrow this list down to only the restaurants
df = df.dropna(subset = ['restaurant'])
df.head()
len(df)

11856

In [11]:
df = df.groupby('restaurant').apply(lambda x: x.sample(600)).reset_index(drop=True)
df['restaurant'].value_counts()

la    600
me    600
as    600
am    600
Name: restaurant, dtype: int64

In [12]:
df.head()

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,restaurant
0,FWSYGI7hmAMx2W0hNbrZzQ,QtaGaJ_ygC5aK5dHwLmw7g,DFeEJf8h04q3KwRVWUsoMQ,5,0,0,0,Gave 5 stars for the arcade/gaming area. Alway...,2018-02-12 19:37:29,am
1,tUUSSuLIegcN00xiyAky2g,Z146DMjYlRbBNB0xRELJXA,woXlprCuowrLJswWere3TQ,5,0,0,0,I went here with a group of friends and it was...,2017-03-15 16:29:49,am
2,p_JOhaSm3d_SKHTdgHmLkQ,myrcQ3h2G04Gv-ANG_oqrg,N8jmyJCk-LjO9o1K9UjxNA,4,6,2,3,"After reading the reviews, I wasn't sure what ...",2011-11-01 23:37:58,am
3,HsyQssCNevOr4tDPOkQDlQ,kM1oPhJEvvc42upOPQwSkw,yNPh5SO-7wr8HPpVCDPbXQ,5,2,2,3,arrogant [ar-uh-guhnt] adjective \n1. making c...,2012-09-18 19:24:04,am
4,z2cNfiLmemGACE_9OOY1rA,m3FleKsUG-1UNqJsJZid9A,XZbuPXdyA0ZtTu3AzqtQhg,5,0,0,0,"YumMMmmm.\n\nWalking in, I was immediately sur...",2008-04-28 22:03:30,am


In [13]:
# the star count is saved as a string, let's make it an integer
df['stars'] = df['stars'].map(lambda x: int(x))

Since I'm working with individual sentences, I need to break the reviews down. However since each sentence is associated with a review and each review with a business, I need to make a new DF.

In [14]:
# adding a tokenized sentence column to the dataframe
df['sents'] = df['text'].map(lambda x: nltk.sent_tokenize(x))

# building a new datafram from this
sent1 = []
for i in df['review_id']:
	sent1.append([i, list(df[df['review_id'] == i]['sents']), int(df[df['review_id'] == i]['stars']), str(df[df['review_id'] == i]['restaurant'].values[0])])
print(sent1[0])
print(sent1[0])


['FWSYGI7hmAMx2W0hNbrZzQ', [['Gave 5 stars for the arcade/gaming area.', 'Always kept clean, bartenders nice and attentive and games kept in order.', "However, I'd give the restaurant itself 2 stars as we stopped eating there.", 'Service is incredibly slow all the time- when they have a few tables or only a couple.', 'Last time I had to flag down the host to let the server know we were seated.', "Then we didn't get our drinks until our food came.", "It's not the first, 5th or 10th time we've had bad experiences at the restaurant so we just gave up.", "Overall tho Dave and busters is great to go to esp on Wednesday's and for happy hour!"]], 5, 'am']
['FWSYGI7hmAMx2W0hNbrZzQ', [['Gave 5 stars for the arcade/gaming area.', 'Always kept clean, bartenders nice and attentive and games kept in order.', "However, I'd give the restaurant itself 2 stars as we stopped eating there.", 'Service is incredibly slow all the time- when they have a few tables or only a couple.', 'Last time I had to flag

In [15]:
sent1[0]

['FWSYGI7hmAMx2W0hNbrZzQ',
 [['Gave 5 stars for the arcade/gaming area.',
   'Always kept clean, bartenders nice and attentive and games kept in order.',
   "However, I'd give the restaurant itself 2 stars as we stopped eating there.",
   'Service is incredibly slow all the time- when they have a few tables or only a couple.',
   'Last time I had to flag down the host to let the server know we were seated.',
   "Then we didn't get our drinks until our food came.",
   "It's not the first, 5th or 10th time we've had bad experiences at the restaurant so we just gave up.",
   "Overall tho Dave and busters is great to go to esp on Wednesday's and for happy hour!"]],
 5,
 'am']

In [17]:
df[df['review_id'] == 'EG2SwUwmv6bimrHpyfxp1A']['restaurant'].values

array([], dtype=object)

In [18]:
df

,review_id,user_id,business_id,stars,useful,funny,cool,text,date,restaurant,sents
0,FWSYGI7hmAMx2W0hNbrZzQ,QtaGaJ_ygC5aK5dHwLmw7g,DFeEJf8h04q3KwRVWUsoMQ,5,0,0,0,Gave 5 stars for the arcade/gaming area. Alway...,2018-02-12 19:37:29,am,"[Gave 5 stars for the arcade/gaming area., Alw..."
1,tUUSSuLIegcN00xiyAky2g,Z146DMjYlRbBNB0xRELJXA,woXlprCuowrLJswWere3TQ,5,0,0,0,I went here with a group of friends and it was...,2017-03-15 16:29:49,am,[I went here with a group of friends and it wa...
2,p_JOhaSm3d_SKHTdgHmLkQ,myrcQ3h2G04Gv-ANG_oqrg,N8jmyJCk-LjO9o1K9UjxNA,4,6,2,3,"After reading the reviews, I wasn't sure what ...",2011-11-01 23:37:58,am,"[After reading the reviews, I wasn't sure what..."
3,HsyQssCNevOr4tDPOkQDlQ,kM1oPhJEvvc42upOPQwSkw,yNPh5SO-7wr8HPpVCDPbXQ,5,2,2,3,arrogant [ar-uh-guhnt] adjective \n1. making c...,2012-09-18 19:24:04,am,[arrogant [ar-uh-guhnt] adjective \n1. making ...
4,z2cNfiLmemGACE_9OOY1rA,m3FleKsUG-1UNqJsJZid9A,XZbuPXdyA0ZtTu3AzqtQhg,5,0,0,0,"YumMMmmm.\n\nWalking in, I was immediately sur...",2008-04-28 22:03:30,am,"[YumMMmmm., Walking in, I was immediately surp..."
...,...,...,...,...,...,...,...,...,...,...,...
2395,OMftIymWnJYDlPz3k_3lfg,RUgbe2W5HjtzPgMqkzSW7w,PBmMvDows5HWIqrEVvNfQQ,5,0,0,0,We first visited this restaurant while my daug...,2016-08-15 15:25:06,me,[We first visited this restaurant while my dau...
2396,zPcYN_WRX5SqriAEfJlI4w,iY4wdIP9aHTXO9zdlShRrQ,ZEZQBsY9mpDjxyXBkvTJMg,1,0,0,0,Zero stars. Booked my brithday here. The waite...,2017-11-19 07:33:52,me,"[Zero stars., Booked my brithday here., The wa..."
2397,x1SfZiFeBFVqBj_IQc_iaA,YfHqK8FFv4tnKlLffvdYdw,ICUCAzUZFiWkYs7iCtNCQg,5,0,0,0,First time here after hearing such great raves...,2015-03-08 03:09:56,me,[First time here after hearing such great rave...
2398,YbK1Toa7BjRJqxZpq5Egdg,reBgsXwB2c73uvLEhs5THg,SMNZS7w9Blmc8TOUZ4oqYg,5,1,0,0,Great offering for a nice healthy meal. Frien...,2015-11-20 19:42:12,me,"[Great offering for a nice healthy meal., Frie..."


In [40]:
sent2 = []

for i in sent1:
    counter = 1
    stars = i[2]
    ID = i[0]
    res = i[3]
    for x in i[1][0]:
        #print([ID, counter, x, stars])
        sent2.append([ID, counter, x, stars, res])
        counter = counter + 1

sent_df = pd.DataFrame(sent2)

In [41]:
print(sent2[0])

['FWSYGI7hmAMx2W0hNbrZzQ', 1, 'Gave 5 stars for the arcade/gaming area.', 5, 'am']


In [42]:
print(sent1[0][1])

[['Gave 5 stars for the arcade/gaming area.', 'Always kept clean, bartenders nice and attentive and games kept in order.', "However, I'd give the restaurant itself 2 stars as we stopped eating there.", 'Service is incredibly slow all the time- when they have a few tables or only a couple.', 'Last time I had to flag down the host to let the server know we were seated.', "Then we didn't get our drinks until our food came.", "It's not the first, 5th or 10th time we've had bad experiences at the restaurant so we just gave up.", "Overall tho Dave and busters is great to go to esp on Wednesday's and for happy hour!"]]


In [43]:
#sent_df['tokenized'] = sent_df[2].map(lambda x: nltk.word_tokenize(x))

In [44]:
sent_df.columns = ['id', 'sent', 'text', 'stars', 'cuisine']

Testing classification on the sentences

In [45]:
len(sent_df)

18514

In [46]:
list(df[df['review_id'] == '8zKt_j1KmxVs3aq_LVeVmg']['sents'])

[['We are from States.',
  'My whole family tried butter chicken,Nihari, chicken shish Kabab and the biryani on special - 2 for $10 and kheer for dessert.',
  'They gave us an amazing customer service and the food was delicious.',
  'I highly recommend to try it!']]

In [47]:
sent_df.to_csv(r'sents.csv')

In [53]:
sent_train = sent_df.groupby('cuisine').apply(lambda x: x.sample(500, random_state=0)).reset_index(drop=True)


In [54]:
sent_train.cuisine.value_counts()

la    500
me    500
as    500
am    500
Name: cuisine, dtype: int64

In [58]:
sent_train.to_csv(r'sent_train.csv') 

In [59]:
#sent_train

,id,sent,text,stars,cuisine
0,6Z4LXZoqY2Rp9LwCL2JWXA,2,We rely on the Cafe quite often after being ou...,4,am
1,S2JET8wLWLAga20TNDlo_w,5,The burgers are smaller in size compared to mo...,5,am
2,drTTGIHuUNIi9_zAg20mtg,20,The New York Strip was wonderfully tender and ...,4,am
3,HqwD-VEjL46jGf168ZAeBg,11,My friend had the soft pretzels and she said t...,4,am
4,KYMi6XViKYHA3ISM9MKwxg,3,I have never eaten here though but would do we...,3,am
...,...,...,...,...,...
1995,cIei1tLnaw0WbHtOlP5Ong,3,The food is still delicious.,5,me
1996,GIEYMKx8kcNk_nAnlXCHwQ,4,"although they some times have music, but still...",3,me
1997,reqq8ufZiz6jxssHXIf34w,25,OMG.,4,me
1998,cKwuT_KaM4eDskOMk_ln4Q,1,This place is awful.,1,me


Annotation guidelines for 'About'

food - 'fd'

service = 'sv'

atmosphere = 'at'

other = None

multiple = 'mt'